## 7.11 类型转换

C++语法有几种不同的类型转换方法：

```cpp
// Example 7.19
int i;
float f;
f = i; // Implicit type conversion
f = (float)i; // C-style type casting
f = float(i); // Constructor-style type casting
f = static_cast<float>(i); // C++ casting operator
```

这些方法具有完全相同的效果。使用哪种方法是编程风格的问题。
下面讨论不同类型转换的时间消耗。

### 有符号 / 无符号转换

```cpp
// Example 7.20
int i;
if ((unsigned int)i < 10) { ...
```

有符号和无符号整数之间的转换，只是编译器以不同的方式解释整数的位。
这里没有溢出检查，代码不需要额外的时间。
这些转换可以自由使用，而不会产生任何性能损失。

### 整型大小转换

```cpp
// Example 7.21
int i;
short int s;
i = s;
```

- 对于有符号整数，通过扩展符号位将整数转换为更长的大小，
- 对于无符号整数，则通过扩展为零位来将整数转换为更长的大小。

如果转换源是算术表达式，这通常需要一个时钟周期。
如果读取存储器中变量的值是时候，来完成大小转换，则通常不需要额外的时间，如例7.22所示。

```cpp
// Example 7.22
short int a[100]; int i, sum = 0;
for (i=0; i<100; i++) sum += a[i];
```

将整数转换为较小的大小，则简单地通过忽略较高的位来完成。不需要溢出检查。 例如：

```cpp
// Example 7.23
int i; short int s;
s = (short int)i;
```

这种转换不需要额外的时间。它只是存储32位整数的低16位。

### 浮点数精度转换

- 在使用浮点寄存器堆栈时，`float`，`double`和`long double`之间的转换不需要额外的时间。
- 当使用XMM寄存器时，它需要2到15个时钟周期（取决于处理器）。

有关寄存器堆栈与XMM寄存器的说明，请参见第32页。 例：

```cpp
// Example 7.24
float a; double b;
a += b;
```

在这个例子中，**如果使用XMM寄存器，转换成本很高**。`a`和`b`应该是相同的类型以避免这种情况。
有关进一步讨论，请参阅第144页。

### 整数到浮点数的转换

将带符号整数转换为浮点数或双精度数据需要4到16个时钟周期，具体取决于处理器和所用寄存器的类型。
**从无符号整数的转换需要更长的时间。
如果没有溢出风险，先将无符号整数转换为有符号整数会更快：**

```cpp
// Example 7.25
unsigned int u; double d;
d = (double)(signed int)u; // Faster, but risk of overflow
```

通过用浮点变量替换整型变量，整数到浮点的转换有时可以避免。例如，待优化代码：

```cpp
// Example 7.26a
float a[100]; int i;
for (i = 0; i < 100; i++) a[i] = 2 * i;
```

**通过创建一个额外的浮点变量，可以避免在此示例中将`i`转换为浮点：**

```cpp
// Example 7.26b
float a[100]; int i; float i2;
for (i = 0, i2 = 0; i < 100; i++, i2 += 2.0f) a[i] = i2;
```

### 浮点到整型的转换

**除非启用SSE2或更高版本的指令集，否则将浮点数转换为整数需要很长时间。通常需要50-100个时钟周期。**
原因是C/C++标准规定使用截断模式，因浮点舍入模式要更改为截断，然后再次改回来。

如果在代码的关键部分中存在浮点到整数转换，那么是有必要做一些事情是。
可能的解决方案是：

- 通过使用不同类型的变量来避免转换。
- 将中间结果存储为浮点数，把转换移出最内层循环。
- 使用64位模式或启用SSE2指令集（需要支持此功能的微处理器）。
- 使用舍入而不是截断，并使用汇编语言创建舍入函数。有关舍入的详细信息，请参见第144页。

### 指针类型转换

指针可以转换为其它不同类型的指针。
同样，一个指针可以转换为一个整数，一个整数也可以转换为一个指针。
整数有足够的位来保存指针是很重要的。

这些转换不会产生任何额外的代码。
转换只是以不同方式解释相同数据位，或绕过语法检查。

当然，这些转换并不安全。程序员有责任确保结果有效。

### 重新解释对象的类型

通过对其地址进行类型转换，可以使编译器将变量或对象视为具有不同的类型：

```cpp
// Example 7.27
float x;
*(int*)&x |= 0x80000000; // Set sign bit of x
```

这里的语法可能有点奇怪。
`x`的地址被类型转换为指向整数的指针，然后解引用该指针以便将`x`作为整数访问。
编译器不会为生成该指针而产生任何额外的代码。
指针被简单地优化掉了，结果是`x`被视为整数。
**但是`&`运算符强制编译器将`x`存储在内存中，而不是寄存器中。**
上面的例子使用`|`运算符来设置`x`的符号位。该运算符只能应用于整数。 它比`x = -abs(x);`更快。

在指针进行类型转换时有许多危险需要注意：

- 这个技巧违反了标准C的严格别名规则（strict aliasing rule），该规则指定不同类型的两个指针不能指向同一个对象（char指针除外）。
优化的编译器可能把浮点数和整数存储在两个不同的寄存器中。
您需要检查编译器是否会按照你预想的去工作。
这里使用`union`更安全，如示例14.23第146页中所示。

- 如果对象被视为比实际更大，那么该技巧将失败。如果int使用的位数多于浮点数，则上面的代码将失败。
（两者都在x86系统中使用32位）。

- 如果访问变量的一部分，例如64位双精度的其中32位，则代码将无法移植到使用大端存储的平台。

- 如果一次访问一部分变量，例如，如果你分两次写入64位的两个32位，那么由于CPU中的存储转发延迟（store forwarding delay），代码可能执行速度比预期慢（参见手册3：“英特尔，AMD和VIA CPU的微体系结构”）。（译者注：存储转发延迟大概意思是先缓存好需要的数据然后一起发送，显然这里会影响性能）


### 常量强制转换

`const_cast`运算符用于规避常量指针的常量限制。
它具有语法检查，因此比C风格的类型转换更安全，且不需要加任何额外的代码。 例如：


```cpp
// Example 7.28
class c1 {
    const int x; // constant data
public:
    c1() : x(0) {}; // constructor initializes x to 0
    void xplus2() { // this function can modify x
        *const_cast<int*>(&x) += 2; // add 2 to x
    }
};
```

这里`const_cast`运算符的作用是删除`x`上的`const`限制。
这是一种规避语法限制的方法，但它不会生成任何额外的代码，也不会占用任何额外的时间。
这是确保某个函数可以修改`x`的有用方法，而其它函数则不能修改。

（译者注：实际未必需要这么麻烦，`(int&)x += 2;`，`const_cast<int&>(x) += 2;`都可以进行该转换）

### 静态强制转换
`static_cast`运算符与C风格的类型转换相同。 例如，它用于将`float`转换为`int`。

### 重新解释型强制转换
`reinterpret_cast`运算符用于指针转换。
它与C语言风格类型转换进行同样的操作，但需要更一点点的语法检查。
它不会产生任何额外的代码。

### 动态强制转换
`dynamic_cast`运算符用于将指向一个类的指针转换为指向另一个类的指针。
它使运行时检查转换是否有效。
例如，当指向基类的指针转换为指向派生类的指针时，它会检查原始指针是否实际指向派生类的对象。
这种检查使`dynamic_cast`比简单的类型转换更耗时，但也更安全。
它可能会捕获到其它方法检测不到的编程错误。

### 转换类的对象
要进行类对象（而不是指向对象的指针）的转换，必须满足下面的条件：
- 定义了构造函数
- 重载了赋值运算符，或者重载了类型转换操作符（指定如何进行转换）

构造函数或重载运算符，与成员函数效率相同。